## Running simulations using OpenMM
`polymerist` and the OpenFF toolkit provide extensive support for running OpenMM simulations,\
due to the relative simplicity of automating these simulations thank to OpenMM's Pythonn API 

### Running a simulation "schedule"
A "schedule" here denotes a serial sequence of simulations, each defined by their own SimulationParameters
Note that subsequent parameter set in the schedule will generate a new directory containing the serailize parametesrs and all files output during the simulation

In [ ]:
...
# TODO: load polymer and parameter sets

In [ ]:
from openff.toolkit import ForceField


forcefield_names : list[str] = ['openff-2.2.0.offxml', 'tip3p.offxml']

forcefield = ForceField(*forcefield_names)
interchange = forcefield.create_interchange(solvated_topology, charge_from_molecules=[polymer])
interchange.box = box_vectors

In [ ]:
from openff.interchange.interop.openmm._positions import to_openmm_positions

from polymerist.mdtools.openmmtools.forcegroups import impose_unique_force_groups
from polymerist.mdtools.openmmtools.execution import run_simulation_schedule


# initialize core OpenMM objects
omm_topology = interchange.to_openmm_topology()
omm_system  = interchange.to_openmm(combine_nonbonded_forces=False)
omm_positions = to_openmm_positions(interchange, include_virtual_sites=True)
impose_unique_force_groups(omm_system) # ensure each Force is separate to enable mapping of energy contributions

# run schedule
schedule = { # simulations will be run in the order they appear here
    'anneal'     : equil_params, 
    'production' : prod_params,
}

history = run_simulation_schedule(
    polymer_outdir, 
    schedule, 
    init_top=omm_topology,
    init_sys=omm_system,
    init_pos=omm_positions,
    return_history=True
)

INFO:polymerist.mdtools.openmmtools.execution:Initializing simulation 1/2 ("anneal")
INFO:polymerist.mdtools.openmmtools.thermo:Created MonteCarloBarostat Force(s) for NPT (Isothermal-isobaric) ensemble
INFO:polymerist.mdtools.openmmtools.preparation:Added MonteCarloBarostat Force to System
INFO:polymerist.mdtools.openmmtools.thermo:Created LangevinMiddleIntegrator for NPT (Isothermal-isobaric) ensemble
INFO:polymerist.mdtools.openmmtools.reporters:Prepared DCDReporter which reports to MD_demo_files/naturalrubber/anneal/anneal_trajectory.dcd
INFO:polymerist.mdtools.openmmtools.reporters:Prepared CheckpointReporter which reports to MD_demo_files/naturalrubber/anneal/anneal_checkpoint.chk
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateReporter which reports to MD_demo_files/naturalrubber/anneal/anneal_state.xml
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateDataReporter which reports to MD_demo_files/naturalrubber/anneal/anneal_state_data.csv
INFO:polymerist.mdt